In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats
from scipy.stats import pearsonr

In [2]:
import pymongo
client=pymongo.MongoClient("192.168.10.240",27017)
db= client['AI_LKJ']

collection_festival=db['festival_period_2023']
data_festival = collection_festival.find()
df_festival = pd.DataFrame(data_festival)

collection_visitor=db['month_visitor_2023']
data_festival = collection_visitor.find()
df_visitor = pd.DataFrame(data_festival)

## 데이터 전처리 과정

### df_festival 결측,중복,이상 제거

In [3]:
df_festival['start_date']= df_festival['date'].str[5:15]
df_festival['end_date']= df_festival['date'].str[18:]
df_festival['start_date']=pd.to_datetime(df_festival['start_date'],format="%Y.%m.%d")
df_festival['end_date']=pd.to_datetime(df_festival['end_date'],format="%Y.%m.%d")
df_festival['festival_period'] = df_festival['end_date'] - df_festival['start_date']

In [4]:
df_festival = df_festival.dropna()

In [5]:
condition_region = "region=='인천'" 
condition_address = "~address.str.contains('인천')"
conditions = f"{condition_address} and {condition_region}"
df_festival.query(conditions).index
df_festival = df_festival.drop(df_festival.query(conditions).index)
df_festival.query(conditions)

,_id,region,title,address,date,start_date,end_date,festival_period


In [6]:
condition_region = "region=='강원도'" 
condition_address = "~address.str.contains('강원도')"
conditions = f"{condition_address} and {condition_region}"
df_festival = df_festival.drop(df_festival.query(conditions).index)
df_festival.query(conditions)

,_id,region,title,address,date,start_date,end_date,festival_period


In [7]:
condition_region = "region=='제주'" 
condition_address = "~address.str.contains('제주')"
conditions = f"{condition_address} and {condition_region}"
df_festival = df_festival.drop(df_festival.query(conditions).index)
df_festival.query(conditions)

,_id,region,title,address,date,start_date,end_date,festival_period


In [8]:
condition_region = "region=='부산'" 
condition_address = "~address.str.contains('부산')"
conditions = f"{condition_address} and {condition_region}"
df_festival = df_festival.drop(df_festival.query(conditions).index)
df_festival.query(conditions)

,_id,region,title,address,date,start_date,end_date,festival_period


In [9]:
condition_region = "region=='서울'" 
condition_address = "~address.str.contains('서울')"
conditions = f"{condition_address} and {condition_region}"
df_festival = df_festival.drop(df_festival.query(conditions).index)
df_festival.query(conditions)

,_id,region,title,address,date,start_date,end_date,festival_period


In [10]:
duplicated_rows = df_festival[df_festival.duplicated(subset=['title', 'address', 'date', 'region'])]
duplicated_rows

,_id,region,title,address,date,start_date,end_date,festival_period
1040,65dbe693bb908e26c802d9b8,부산,2023 해운대카운트다운&해맞이축제,부산광역시 해운대구 중동2로 11,일정 : 2023.01.01 ~ 2023.01.01,2023-01-01,2023-01-01,0 days
1041,65dbe693bb908e26c802d9b9,부산,2023 계묘년 해맞이행사,부산광역시 남구 오륙도로 137,일정 : 2023.01.01 ~ 2023.01.01,2023-01-01,2023-01-01,0 days
1054,65dbe693bb908e26c802d9c6,부산,동래읍성역사축제,부산광역시 동래구 문화로 80(명륜동),일정 : 2023.10.13 ~ 2023.10.15,2023-10-13,2023-10-15,2 days
3149,65dbe694bb908e26c802e1f5,제주,휴애리 유채꽃 축제,제주특별자치도 서귀포시 남원읍 신례동로 256,일정 : 2023.01.01 ~ 2023.03.31,2023-01-01,2023-03-31,89 days
4115,65dbe694bb908e26c802e5bb,강원도,태백산 눈축제,강원도 태백시 장성1길 171,일정 : 2023.01.01 ~ 2023.01.01,2023-01-01,2023-01-01,0 days
4120,65dbe694bb908e26c802e5c0,강원도,제25회 원주한지문화제,강원도 원주시 한지공원길 151,일정 : 2023.05.05 ~ 2023.05.14,2023-05-05,2023-05-14,9 days
4124,65dbe694bb908e26c802e5c4,강원도,제16회 용수골꽃양귀비축제,강원도 원주시 한지공원길 151,일정 : 2023.05.23 ~ 2023.06.11,2023-05-23,2023-06-11,19 days
4126,65dbe694bb908e26c802e5c6,강원도,제21회 장미축제,강원도 원주시 장미공원길 67-16,일정 : 2023.05.30 ~ 2023.06.01,2023-05-30,2023-06-01,2 days
4129,65dbe694bb908e26c802e5c9,강원도,제13회 남한강축제,강원도 원주시 부론면 앙암로 1234,일정 : 2023.07.29 ~ 2023.07.30,2023-07-29,2023-07-30,1 days
4131,65dbe694bb908e26c802e5cb,강원도,치악산복숭아축제,강원도 원주시 단구로 170,일정 : 2023.08.01 ~ 2023.08.01,2023-08-01,2023-08-01,0 days


In [11]:
df_festival=df_festival.drop_duplicates(subset=['title','address','date','region'])
df_festival.duplicated(subset=['title','address','date','region']).sum()

0

In [12]:
condition_region = "region=='인천'" 
df_festival.query(condition_region).sort_values(by='date')

,_id,region,title,address,date,start_date,end_date,festival_period
5,65dbe693bb908e26c802d5ad,인천,2023 자유공원 벚꽃축제,인천광역시 중구 제물량로232번길 46,일정 : 2023.04.08 ~ 2023.04.08,2023-04-08,2023-04-08,0 days
6,65dbe693bb908e26c802d5ae,인천,계양산국악제,인천광역시 계양구 주부토로 570,일정 : 2023.04.21 ~ 2023.04.22,2023-04-21,2023-04-22,1 days
7,65dbe693bb908e26c802d5af,인천,2023 가족의 달 어린이축제,인천광역시 중구 제물량로 238,일정 : 2023.05.06 ~ 2023.05.07,2023-05-06,2023-05-07,1 days
3,65dbe693bb908e26c802d5ab,인천,제34회 화도진축제,인천광역시 동구 참외전로 125 (동인천역 광장),일정 : 2023.05.19 ~ 2023.05.20,2023-05-19,2023-05-20,1 days
8,65dbe693bb908e26c802d5b0,인천,아라뱃길 카약축제,인천광역시 서구 정서진1로 41,일정 : 2023.06.17 ~ 2023.06.17,2023-06-17,2023-06-17,0 days
0,65dbe693bb908e26c802d5a8,인천,인천펜타포트 음악축제,인천광역시 연수구 센트럴로 350,일정 : 2023.08.04 ~ 2023.08.06,2023-08-04,2023-08-06,2 days
1,65dbe693bb908e26c802d5a9,인천,인천펜타포트락페스티벌,인천광역시 연수구 센트럴로 350,일정 : 2023.08.04 ~ 2023.08.06,2023-08-04,2023-08-06,2 days
9,65dbe693bb908e26c802d5b1,인천,무의도 춤축제,인천광역시 중구 신포로27번길 80,일정 : 2023.08.05 ~ 2023.08.05,2023-08-05,2023-08-05,0 days
11,65dbe693bb908e26c802d5b3,인천,INK 콘서트,인천광역시 미추홀구 매소홀로 618,일정 : 2023.09.09 ~ 2023.09.09,2023-09-09,2023-09-09,0 days
12,65dbe693bb908e26c802d5b4,인천,제23회소래포구축제,"인천광역시 남동구 아암대로 1550, (논현동)",일정 : 2023.09.15 ~ 2023.09.17,2023-09-15,2023-09-17,2 days


In [13]:
df_festival = df_festival.drop([0,14])

In [14]:
condition_region = "region=='강원도'" 
df_festival.query(condition_region).sort_values(by='date')

,_id,region,title,address,date,start_date,end_date,festival_period
4112,65dbe694bb908e26c802e5b8,강원도,애니메이션박물관 제17회 기획전 세상을 바꾼 불꽃 태일이,강원도 춘천시 서면 박사로 854 애니메이션박물관,일정 : 2022.05.03 ~ 2023.04.16,2022-05-03,2023-04-16,348 days
4113,65dbe694bb908e26c802e5b9,강원도,강원대학교 산학협력단 K-Digital Training 제2기 교육생 모집,강원도 춘천시 강원대학길 1,일정 : 2022.10.04 ~ 2023.03.13,2022-10-04,2023-03-13,160 days
4114,65dbe694bb908e26c802e5ba,강원도,평창송어축제,강원도 평창군 진부면 경강로 3562,일정 : 2022.12.30 ~ 2023.01.29,2022-12-30,2023-01-29,30 days
4110,65dbe694bb908e26c802e5b6,강원도,태백산 눈축제,강원도 태백시 장성1길 171,일정 : 2023.01.01 ~ 2023.01.01,2023-01-01,2023-01-01,0 days
4116,65dbe694bb908e26c802e5bc,강원도,대관령눈꽃축제,강원도 평창군 대관령면 사부랑길224-2,일정 : 2023.01.13 ~ 2023.01.22,2023-01-13,2023-01-22,9 days
4117,65dbe694bb908e26c802e5bd,강원도,삼척정월대보름제,강원도 강원도 삼척시청 엑스포로 45,일정 : 2023.02.03 ~ 2023.02.05,2023-02-03,2023-02-05,2 days
4118,65dbe694bb908e26c802e5be,강원도,삼척맹방유채꽃행사,강원도 삼척시 근덕면 삼척로 3916-112(마을회관),일정 : 2023.04.07 ~ 2023.04.16,2023-04-07,2023-04-16,9 days
4119,65dbe694bb908e26c802e5bf,강원도,제25회 원주한지문화제,강원도 원주시 한지공원길 151,일정 : 2023.05.05 ~ 2023.05.14,2023-05-05,2023-05-14,9 days
4121,65dbe694bb908e26c802e5c1,강원도,삼척장미축제,강원도 강원도 삼척시청 오십천로 536,일정 : 2023.05.20 ~ 2023.05.24,2023-05-20,2023-05-24,4 days
4122,65dbe694bb908e26c802e5c2,강원도,황영조 국제 마라톤 대회,강원도 강원도 삼척시청 엑스포로 45,일정 : 2023.05.21 ~ 2023.05.21,2023-05-21,2023-05-21,0 days


In [15]:
df_festival = df_festival.drop([4113])

In [16]:
condition_region = "region=='제주'" 
df_festival.query(condition_region).sort_values(by='date')

,_id,region,title,address,date,start_date,end_date,festival_period
3101,65dbe694bb908e26c802e1c5,제주,제주 다이노스 어라이브,제주특별자치도 서귀포시 안덕면 녹차분재로 218,일정 : 2022.08.10 ~ 2023.08.09,2022-08-10,2023-08-09,364 days
3148,65dbe694bb908e26c802e1f4,제주,빛의 벙커 ’세잔프로방스의빛/칸딘스키’ 전,제주특별자치도 서귀포시 선산읍 서성이로 1168번길 89-17,일정 : 2022.11.04 ~ 2023.10.15,2022-11-04,2023-10-15,345 days
3112,65dbe694bb908e26c802e1d0,제주,제30회 성산일출축제,제주특별자치도 서귀포시 성산읍 일출로 284-12,일정 : 2022.12.31 ~ 2023.01.01,2022-12-31,2023-01-01,1 days
3080,65dbe694bb908e26c802e1b0,제주,휴애리 유채꽃 축제,제주특별자치도 서귀포시 남원읍 신례동로 256,일정 : 2023.01.01 ~ 2023.03.31,2023-01-01,2023-03-31,89 days
3113,65dbe694bb908e26c802e1d1,제주,한림공원 수선화축제,제주특별자치도 제주시 한림읍 한림로 300,일정 : 2023.01.21 ~ 2023.02.19,2023-01-21,2023-02-19,29 days
...,...,...,...,...,...,...,...,...
3110,65dbe694bb908e26c802e1ce,제주,제29회 제주 성읍마을 전통민속재현축제,제주특별자치도 서귀포시 표선면 성읍정의현로 30,일정 : 2023.11.03 ~ 2023.11.05,2023-11-03,2023-11-05,2 days
3111,65dbe694bb908e26c802e1cf,제주,2023 제주시 소상공인 한마음 박람회,제주특별자치도 제주시 오일장서길 26,일정 : 2023.11.10 ~ 2023.11.11,2023-11-10,2023-11-11,1 days
3147,65dbe694bb908e26c802e1f3,제주,서문반짝마켓,제주특별자치도 제주시 서문로4길 13-2,일정 : 2023.11.10 ~ 2023.12.02,2023-11-10,2023-12-02,22 days
3219,65dbe694bb908e26c802e23b,제주,휴애리 동백축제,제주특별자치도 서귀포시 남원읍 신례동로 256,일정 : 2023.11.14 ~ 2024.01.31,2023-11-14,2024-01-31,78 days


In [17]:
df_festival = df_festival.drop([3115,3116,3120,3159,3160,3164,3166,3195,3183,3214,3207,3204])

In [18]:
condition_region = "region=='부산'" 
df_festival.query(condition_region).sort_values(by='date')

,_id,region,title,address,date,start_date,end_date,festival_period
1026,65dbe693bb908e26c802d9aa,부산,2023 해운대카운트다운&해맞이축제,부산광역시 해운대구 중동2로 11,일정 : 2023.01.01 ~ 2023.01.01,2023-01-01,2023-01-01,0 days
1027,65dbe693bb908e26c802d9ab,부산,2023 계묘년 해맞이행사,부산광역시 남구 오륙도로 137,일정 : 2023.01.01 ~ 2023.01.01,2023-01-01,2023-01-01,0 days
1042,65dbe693bb908e26c802d9ba,부산,2023 지금여기 금정 관광후기 이벤트,부산광역시 금정구 중앙대로 1777,일정 : 2023.02.01 ~ 2023.11.30,2023-02-01,2023-11-30,302 days
1043,65dbe693bb908e26c802d9bb,부산,제38회 해운대 달맞이온천축제,부산광역시 해운대구 중동2로 11,일정 : 2023.02.03 ~ 2023.02.05,2023-02-03,2023-02-05,2 days
1044,65dbe693bb908e26c802d9bc,부산,2023 정월대보름 달맞이축제,부산광역시 남구 분포로 66-40,일정 : 2023.02.05 ~ 2023.02.05,2023-02-05,2023-02-05,0 days
1045,65dbe693bb908e26c802d9bd,부산,낙동강유채꽃축제,부산광역시 강서구 공항로 1246(대저1동),일정 : 2023.04.08 ~ 2023.04.16,2023-04-08,2023-04-16,8 days
1046,65dbe693bb908e26c802d9be,부산,광안리어방축제,부산광역시 수영구 남천동로100,일정 : 2023.05.12 ~ 2023.05.14,2023-05-12,2023-05-14,2 days
1047,65dbe693bb908e26c802d9bf,부산,제18회 해운대 모래축제,부산광역시 해운대구 중동2로 11,일정 : 2023.05.19 ~ 2023.05.22,2023-05-19,2023-05-22,3 days
1029,65dbe693bb908e26c802d9ad,부산,부산원도심골목길축제,부산광역시 동구 증산북로 18-11,일정 : 2023.05.20 ~ 2023.05.21,2023-05-20,2023-05-21,1 days
1048,65dbe693bb908e26c802d9c0,부산,제18회 국제매직페스티벌,부산광역시 해운대구 중동2로 11,일정 : 2023.07.01 ~ 2023.07.02,2023-07-01,2023-07-02,1 days


In [19]:
df_festival = df_festival.drop([1030,1037])

In [20]:
condition_region = "region=='서울'" 
df_festival.query(condition_region).sort_values(by='date')

,_id,region,title,address,date,start_date,end_date,festival_period
2058,65dbe693bb908e26c802ddb2,서울,제20회 강북구 3·1독립운동 기념행사,서울특별시 강북구 삼양로173길 107-12,일정 : 2023.03.01 ~ 2023.03.01,2023-03-01,2023-03-01,0 days
2059,65dbe693bb908e26c802ddb3,서울,제1회 중곡2동 긴고랑 왕벚꽃축제,서울특별시 광진구 능동로 346(중곡동),일정 : 2023.04.01 ~ 2023.04.01,2023-04-01,2023-04-01,0 days
2060,65dbe693bb908e26c802ddb4,서울,동대문구 봄꽃축제,서울특별시 동대문구 천호대로 145,일정 : 2023.04.01 ~ 2023.04.02,2023-04-01,2023-04-02,1 days
2061,65dbe693bb908e26c802ddb5,서울,제1회 중곡3동 벚꽃축제,서울특별시 광진구 동일로76가길 28(중곡동),일정 : 2023.04.02 ~ 2023.04.02,2023-04-02,2023-04-02,0 days
2062,65dbe693bb908e26c802ddb6,서울,석촌호수벚꽃축제,서울특별시 송파구 석촌호수로 197,일정 : 2023.04.05 ~ 2023.04.09,2023-04-05,2023-04-09,4 days
...,...,...,...,...,...,...,...,...
2114,65dbe693bb908e26c802ddea,서울,서초골 축제,서울특별시 서초구 남부순환로 2406,일정 : 2023.11.01 ~ 2023.11.30,2023-11-01,2023-11-30,29 days
2115,65dbe693bb908e26c802ddeb,서울,2023 모두의 축제: 청춘 관악,서울특별시 관악구 관천로 53,일정 : 2023.11.04 ~ 2023.11.04,2023-11-04,2023-11-04,0 days
2116,65dbe693bb908e26c802ddec,서울,개천절 대제전,서울특별시 종로구 인왕산로 22,일정 : 2023.11.15 ~ 2023.11.15,2023-11-15,2023-11-15,0 days
2117,65dbe693bb908e26c802dded,서울,2023년 송년의 밤 특별 콘서트,서울특별시 강서구 가로공원로 195,일정 : 2023.12.12 ~ 2023.12.12,2023-12-12,2023-12-12,0 days


In [21]:
df_festival = df_festival.drop([2057])

In [22]:
df_festival.loc[2065,'end_date']=df_festival.loc[2065,'end_date'].replace(year=2023)
df_festival.loc[2065,'start_date']=df_festival.loc[2065,'start_date'].replace(year=2023)
df_festival['festival_period'] = df_festival['end_date'] - df_festival['start_date']

In [23]:
df_festival.loc[3169,'end_date']=df_festival.loc[3169,'end_date'].replace(day=26)
df_festival.loc[3169,'start_date']=df_festival.loc[3169,'start_date'].replace(day=18)
df_festival['festival_period'] = df_festival['end_date'] - df_festival['start_date']

### df_visitor 결측,중복,이상 제거

In [24]:
pivot_visitor = pd.pivot_table(data=df_visitor,index="지역구분",columns = '기준년월',values='방문자_수', aggfunc='sum')
pivot_visitor.loc['total',202301]=pivot_visitor[202301].sum()
pivot_visitor.loc['total',202302]=pivot_visitor[202302].sum()
pivot_visitor.loc['total',202303]=pivot_visitor[202303].sum()
pivot_visitor.loc['total',202304]=pivot_visitor[202304].sum()
pivot_visitor.loc['total',202305]=pivot_visitor[202305].sum()
pivot_visitor.loc['total',202306]=pivot_visitor[202306].sum()
pivot_visitor.loc['total',202307]=pivot_visitor[202307].sum()
pivot_visitor.loc['total',202308]=pivot_visitor[202308].sum()
pivot_visitor.loc['total',202309]=pivot_visitor[202309].sum()
pivot_visitor.loc['total',202310]=pivot_visitor[202310].sum()
pivot_visitor.loc['total',202311]=pivot_visitor[202311].sum()
pivot_visitor.loc['total',202312]=pivot_visitor[202312].sum()
pivot_visitor.loc['강원','total']=pivot_visitor.loc['강원',:].sum()
pivot_visitor.loc['부산','total']=pivot_visitor.loc['부산',:].sum()
pivot_visitor.loc['서울','total']=pivot_visitor.loc['서울',:].sum()
pivot_visitor.loc['인천','total']=pivot_visitor.loc['인천',:].sum()
pivot_visitor.loc['제주','total']=pivot_visitor.loc['제주',:].sum()
pivot_visitor.loc['total','total']=pivot_visitor.loc['total',:].sum()

In [25]:
pivot_visitor

기준년월,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,total
지역구분,,,,,,,,,,,,,
강원,12613168.0,11183755.0,12150911.0,11164195.0,12499622.0,12291416.0,14682347.0,17103945.0,13520176.0,13928972.0,9878631.0,11320438.0,1.523376e+08
부산,13180063.0,11238505.0,13159913.0,11489345.0,12455221.0,11702524.0,12825366.0,13322368.0,13666837.0,12641711.0,11626722.0,12946350.0,1.502549e+08
서울,47255715.0,47090565.0,54980858.0,45025765.0,49506461.0,47768488.0,50292484.0,49277340.0,51928425.0,51118153.0,49451765.0,52547645.0,5.962437e+08
인천,14321833.0,13187354.0,15620369.0,13830096.0,15906026.0,14338822.0,15492589.0,15449316.0,16422889.0,16131144.0,14341875.0,15642889.0,1.806852e+08
제주,5377591.0,4935915.0,5274889.0,5123197.0,5208883.0,5059521.0,5336175.0,5982112.0,4788081.0,5614039.0,4672499.0,4638646.0,6.201155e+07
total,92748370.0,87636094.0,101186940.0,86632598.0,95576213.0,91160771.0,98628961.0,101135081.0,100326408.0,99434019.0,89971492.0,97095968.0,1.141533e+09


### 월별 축제 개최 수

In [26]:
df_festival_month = pd.DataFrame(data= df_festival[['region','start_date','end_date']])
df_festival_month

,region,start_date,end_date
1,인천,2023-08-04,2023-08-06
2,인천,2023-10-14,2023-10-15
3,인천,2023-05-19,2023-05-20
5,인천,2023-04-08,2023-04-08
6,인천,2023-04-21,2023-04-22
...,...,...,...
4141,강원도,2023-10-01,2023-10-01
4144,강원도,2023-10-06,2023-10-07
4146,강원도,2023-10-06,2023-10-08
4147,강원도,2023-10-21,2023-10-22


In [27]:
import pandas as pd
from datetime import datetime

date_start = '2023-01-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-01-31'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'01']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01
1026,부산,2023-01-01,2023-01-01,1.0
1027,부산,2023-01-01,2023-01-01,1.0
3080,제주,2023-01-01,2023-03-31,1.0
3101,제주,2022-08-10,2023-08-09,1.0
3112,제주,2022-12-31,2023-01-01,1.0
3113,제주,2023-01-21,2023-02-19,1.0
3114,제주,2023-01-30,2023-11-05,1.0
3148,제주,2022-11-04,2023-10-15,1.0
4110,강원도,2023-01-01,2023-01-01,1.0
4112,강원도,2022-05-03,2023-04-16,1.0


In [28]:
date_start = '2023-02-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-02-28'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'02']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02
1042,부산,2023-02-01,2023-11-30,NaN,1.0
1043,부산,2023-02-03,2023-02-05,NaN,1.0
1044,부산,2023-02-05,2023-02-05,NaN,1.0
3080,제주,2023-01-01,2023-03-31,1.0,1.0
3101,제주,2022-08-10,2023-08-09,1.0,1.0
3113,제주,2023-01-21,2023-02-19,1.0,1.0
3114,제주,2023-01-30,2023-11-05,1.0,1.0
3148,제주,2022-11-04,2023-10-15,1.0,1.0
3150,제주,2023-02-02,2023-02-04,NaN,1.0
3151,제주,2023-02-11,2023-03-12,NaN,1.0


In [29]:
date_start = '2023-03-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-03-31'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'03']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0
2058,서울,2023-03-01,2023-03-01,NaN,NaN,1.0
3080,제주,2023-01-01,2023-03-31,1.0,1.0,1.0
3081,제주,2023-03-09,2023-03-12,NaN,NaN,1.0
3082,제주,2023-03-25,2023-04-15,NaN,NaN,1.0
3083,제주,2023-03-26,2023-03-26,NaN,NaN,1.0
3101,제주,2022-08-10,2023-08-09,1.0,1.0,1.0
3114,제주,2023-01-30,2023-11-05,1.0,1.0,1.0
3118,제주,2023-03-18,2023-04-22,NaN,NaN,1.0
3119,제주,2023-03-25,2023-03-26,NaN,NaN,1.0


In [30]:
date_start = '2023-04-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-04-30'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'04']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04
5,인천,2023-04-08,2023-04-08,NaN,NaN,NaN,1.0
6,인천,2023-04-21,2023-04-22,NaN,NaN,NaN,1.0
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0,1.0
1045,부산,2023-04-08,2023-04-16,NaN,NaN,NaN,1.0
2059,서울,2023-04-01,2023-04-01,NaN,NaN,NaN,1.0
2060,서울,2023-04-01,2023-04-02,NaN,NaN,NaN,1.0
2061,서울,2023-04-02,2023-04-02,NaN,NaN,NaN,1.0
2062,서울,2023-04-05,2023-04-09,NaN,NaN,NaN,1.0
2063,서울,2023-04-08,2023-04-19,NaN,NaN,NaN,1.0
2064,서울,2023-04-22,2023-04-22,NaN,NaN,NaN,1.0


In [31]:
date_start = '2023-05-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-05-31'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'05']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05
3,인천,2023-05-19,2023-05-20,NaN,NaN,NaN,NaN,1.0
7,인천,2023-05-06,2023-05-07,NaN,NaN,NaN,NaN,1.0
1029,부산,2023-05-20,2023-05-21,NaN,NaN,NaN,NaN,1.0
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0,1.0,1.0
1046,부산,2023-05-12,2023-05-14,NaN,NaN,NaN,NaN,1.0
1047,부산,2023-05-19,2023-05-22,NaN,NaN,NaN,NaN,1.0
2054,서울,2023-05-13,2023-05-28,NaN,NaN,NaN,NaN,1.0
2055,서울,2023-05-06,2023-05-07,NaN,NaN,NaN,NaN,1.0
2066,서울,2023-04-29,2023-06-30,NaN,NaN,NaN,1.0,1.0
2067,서울,2023-05-04,2023-05-04,NaN,NaN,NaN,NaN,1.0


In [32]:
date_start = '2023-06-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-06-30'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'06']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05,06
8,인천,2023-06-17,2023-06-17,NaN,NaN,NaN,NaN,NaN,1.0
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0,1.0,1.0,1.0
2066,서울,2023-04-29,2023-06-30,NaN,NaN,NaN,1.0,1.0,1.0
2074,서울,2023-06-10,2023-06-10,NaN,NaN,NaN,NaN,NaN,1.0
2075,서울,2023-06-17,2023-06-18,NaN,NaN,NaN,NaN,NaN,1.0
2076,서울,2023-06-17,2023-06-17,NaN,NaN,NaN,NaN,NaN,1.0
2077,서울,2023-06-30,2023-07-01,NaN,NaN,NaN,NaN,NaN,1.0
3087,제주,2023-05-06,2023-10-28,NaN,NaN,NaN,NaN,1.0,1.0
3089,제주,2023-05-11,2023-11-30,NaN,NaN,NaN,NaN,1.0,1.0
3092,제주,2023-05-25,2023-06-25,NaN,NaN,NaN,NaN,1.0,1.0


In [33]:
date_start = '2023-07-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-07-31'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'07']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05,06,07
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0,1.0,1.0,1.0,1.0
1048,부산,2023-07-01,2023-07-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2077,서울,2023-06-30,2023-07-01,NaN,NaN,NaN,NaN,NaN,1.0,1.0
2078,서울,2023-07-01,2023-07-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3087,제주,2023-05-06,2023-10-28,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3089,제주,2023-05-11,2023-11-30,NaN,NaN,NaN,NaN,1.0,1.0,1.0
3095,제주,2023-06-15,2023-07-03,NaN,NaN,NaN,NaN,NaN,1.0,1.0
3096,제주,2023-07-01,2023-11-05,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3101,제주,2022-08-10,2023-08-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3104,제주,2023-07-28,2023-10-31,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [34]:
date_start = '2023-08-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-08-31'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'08']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05,06,07,08
1,인천,2023-08-04,2023-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,인천,2023-08-05,2023-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1031,부산,2023-08-01,2023-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1049,부산,2023-08-04,2023-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1050,부산,2023-08-12,2023-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2079,서울,2023-08-18,2023-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2080,서울,2023-08-26,2023-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2081,서울,2023-08-26,2023-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3087,제주,2023-05-06,2023-10-28,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0


In [35]:
date_start = '2023-09-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-09-30'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'09']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05,06,07,08,09
11,인천,2023-09-09,2023-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12,인천,2023-09-15,2023-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
13,인천,2023-09-17,2023-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
15,인천,2023-09-22,2023-09-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
16,인천,2023-09-26,2024-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1051,부산,2023-09-16,2023-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2079,서울,2023-08-18,2023-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
2082,서울,2023-09-01,2023-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2083,서울,2023-09-01,2023-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [36]:
date_start = '2023-10-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-10-31'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'10']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05,06,07,08,09,10
2,인천,2023-10-14,2023-10-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
16,인천,2023-09-26,2024-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
17,인천,2023-10-05,2023-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
18,인천,2023-10-07,2023-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
19,인천,2023-10-21,2023-10-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4140,강원도,2023-10-01,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4141,강원도,2023-10-01,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4144,강원도,2023-10-06,2023-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4146,강원도,2023-10-06,2023-10-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
pd.pivot_table(data= df_festival_month, index=

In [37]:
date_start = '2023-11-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-11-30'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'11']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05,06,07,08,09,10,11
16,인천,2023-09-26,2024-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
1028,부산,2023-11-04,2023-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1035,부산,2023-10-13,2024-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
1042,부산,2023-02-01,2023-11-30,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1064,부산,2023-11-03,2023-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1065,부산,2023-11-04,2023-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2102,서울,2023-10-13,2023-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
2114,서울,2023-11-01,2023-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2115,서울,2023-11-04,2023-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2116,서울,2023-11-15,2023-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [38]:
date_start = '2023-12-01'
date_start = datetime.strptime(date_start, '%Y-%m-%d')

date_end = '2023-12-31'
date_end = datetime.strptime(date_end, '%Y-%m-%d')
# start_date가 specified_date보다 크거나 같은 행들을 필터링
df_festival_month.loc[df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)].index,'12']=1
df_festival_month[(df_festival_month['end_date'] >= date_start) & (df_festival_month['start_date'] <= date_end)]

,region,start_date,end_date,01,02,03,04,05,06,07,08,09,10,11,12
16,인천,2023-09-26,2024-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
21,인천,2023-12-31,2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1035,부산,2023-10-13,2024-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
1038,부산,2023-12-31,2024-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1066,부산,2023-12-01,2023-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1067,부산,2023-12-01,2024-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1068,부산,2023-12-02,2024-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1069,부산,2023-12-31,2024-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2056,서울,2023-12-22,2023-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2117,서울,2023-12-12,2023-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [129]:
pivot_festival_month=pd.pivot_table(data=df_festival_month, index='region', values=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'],aggfunc='sum')
# 지역별 축제 갯수를 월 기준으로 나눠서 표시
pivot_visitor = pd.pivot_table(data=df_visitor,index="지역구분",columns = '기준년월',values='방문자_수', aggfunc='sum')

## 데이터 분석
- 가설 : 축제가 많은 지역일 수록 방문자 수가 증가한다. 아니면 관심을 많이 받는다.
- 분석 데이터 : (pivot_festival_month의 축제 개수값, 해당 지역의 방문자 수) 

### data

In [132]:
df_visitor

,_id,기준년월,방문자_수,지역구분
0,65dd76f9bb908e26c8034161,202301,14321833,인천
1,65dd76f9bb908e26c8034162,202302,13187354,인천
2,65dd76f9bb908e26c8034163,202303,15620369,인천
3,65dd76f9bb908e26c8034164,202304,13830096,인천
4,65dd76f9bb908e26c8034165,202305,15906026,인천
5,65dd76f9bb908e26c8034166,202306,14338822,인천
6,65dd76f9bb908e26c8034167,202307,15492589,인천
7,65dd76f9bb908e26c8034168,202308,15449316,인천
8,65dd76f9bb908e26c8034169,202309,16422889,인천
9,65dd76f9bb908e26c803416a,202310,16131144,인천


In [110]:
pivot_festival_month # 월별 개최 수

,01,02,03,04,05,06,07,08,09,10,11,12
region,,,,,,,,,,,,
강원도,4.0,2.0,1.0,2.0,5.0,2.0,2.0,1.0,5.0,5.0,1.0,0.0
부산,2.0,3.0,1.0,2.0,4.0,1.0,2.0,4.0,2.0,18.0,5.0,6.0
서울,0.0,0.0,1.0,8.0,10.0,5.0,2.0,3.0,11.0,23.0,4.0,2.0
인천,0.0,0.0,0.0,2.0,2.0,1.0,0.0,2.0,5.0,6.0,1.0,2.0
제주,6.0,8.0,17.0,22.0,23.0,29.0,25.0,28.0,32.0,34.0,15.0,4.0


In [131]:
pivot_visitor=pivot_visitor.loc[:"제주",:202312] # 월별 방문자 수
pivot_visitor

기준년월,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312
지역구분,,,,,,,,,,,,
강원,12613168,11183755,12150911,11164195,12499622,12291416,14682347,17103945,13520176,13928972,9878631,11320438
부산,13180063,11238505,13159913,11489345,12455221,11702524,12825366,13322368,13666837,12641711,11626722,12946350
서울,47255715,47090565,54980858,45025765,49506461,47768488,50292484,49277340,51928425,51118153,49451765,52547645
인천,14321833,13187354,15620369,13830096,15906026,14338822,15492589,15449316,16422889,16131144,14341875,15642889
제주,5377591,4935915,5274889,5123197,5208883,5059521,5336175,5982112,4788081,5614039,4672499,4638646


In [112]:
pivot_festival_month.index

Index(['강원도', '부산', '서울', '인천', '제주'], dtype='object', name='region')

In [113]:
pivot_festival_month = pivot_festival_month.rename(index={'강원도': "강원"})

In [114]:
pivot_festival_month.columns

Index(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], dtype='object')

In [117]:
pivot_festival_month = pivot_festival_month.rename(columns={'01': 202301,'02': 202302,'03': 202303,'04': 202304,'05': 202305,'06': 202306,'07': 202307,'08': 202308,'09': 202309,'10': 202310,'11': 202311,'12': 202312})

In [146]:
df_festival_month.query("region=='인천'")['01'].sum()

0.0

In [203]:
df_festival_months = pd.DataFrame(df_festival_month['region'].unique())
df_festival_months = df_festival_months.rename(columns={0:'지역구분'})
df_festival_months.loc[0,202301]=df_festival_month.query("region=='인천'")['01'].sum()
df_festival_months.loc[1,202301]=df_festival_month.query("region=='부산'")['01'].sum()
df_festival_months.loc[2,202301]=df_festival_month.query("region=='서울'")['01'].sum()
df_festival_months.loc[3,202301]=df_festival_month.query("region=='제주'")['01'].sum()
df_festival_months.loc[4,202301]=df_festival_month.query("region=='강원도'")['01'].sum()
df_festival_months.loc[0,202302]=df_festival_month.query("region=='인천'")['02'].sum()
df_festival_months.loc[1,202302]=df_festival_month.query("region=='부산'")['02'].sum()
df_festival_months.loc[2,202302]=df_festival_month.query("region=='서울'")['02'].sum()
df_festival_months.loc[3,202302]=df_festival_month.query("region=='제주'")['02'].sum()
df_festival_months.loc[4,202302]=df_festival_month.query("region=='강원도'")['02'].sum()
df_festival_months.loc[0,202303]=df_festival_month.query("region=='인천'")['03'].sum()
df_festival_months.loc[1,202303]=df_festival_month.query("region=='부산'")['03'].sum()
df_festival_months.loc[2,202303]=df_festival_month.query("region=='서울'")['03'].sum()
df_festival_months.loc[3,202303]=df_festival_month.query("region=='제주'")['03'].sum()
df_festival_months.loc[4,202303]=df_festival_month.query("region=='강원도'")['03'].sum()
df_festival_months.loc[0,202304]=df_festival_month.query("region=='인천'")['04'].sum()
df_festival_months.loc[1,202304]=df_festival_month.query("region=='부산'")['04'].sum()
df_festival_months.loc[2,202304]=df_festival_month.query("region=='서울'")['04'].sum()
df_festival_months.loc[3,202304]=df_festival_month.query("region=='제주'")['04'].sum()
df_festival_months.loc[4,202304]=df_festival_month.query("region=='강원도'")['04'].sum()
df_festival_months.loc[0,202305]=df_festival_month.query("region=='인천'")['05'].sum()
df_festival_months.loc[1,202305]=df_festival_month.query("region=='부산'")['05'].sum()
df_festival_months.loc[2,202305]=df_festival_month.query("region=='서울'")['05'].sum()
df_festival_months.loc[3,202305]=df_festival_month.query("region=='제주'")['05'].sum()
df_festival_months.loc[4,202305]=df_festival_month.query("region=='강원도'")['05'].sum()
df_festival_months.loc[0,202306]=df_festival_month.query("region=='인천'")['06'].sum()
df_festival_months.loc[1,202306]=df_festival_month.query("region=='부산'")['06'].sum()
df_festival_months.loc[2,202306]=df_festival_month.query("region=='서울'")['06'].sum()
df_festival_months.loc[3,202306]=df_festival_month.query("region=='제주'")['06'].sum()
df_festival_months.loc[4,202306]=df_festival_month.query("region=='강원도'")['06'].sum()
df_festival_months.loc[0,202307]=df_festival_month.query("region=='인천'")['07'].sum()
df_festival_months.loc[1,202307]=df_festival_month.query("region=='부산'")['07'].sum()
df_festival_months.loc[2,202307]=df_festival_month.query("region=='서울'")['07'].sum()
df_festival_months.loc[3,202307]=df_festival_month.query("region=='제주'")['07'].sum()
df_festival_months.loc[4,202307]=df_festival_month.query("region=='강원도'")['07'].sum()
df_festival_months.loc[0,202308]=df_festival_month.query("region=='인천'")['08'].sum()
df_festival_months.loc[1,202308]=df_festival_month.query("region=='부산'")['08'].sum()
df_festival_months.loc[2,202308]=df_festival_month.query("region=='서울'")['08'].sum()
df_festival_months.loc[3,202308]=df_festival_month.query("region=='제주'")['08'].sum()
df_festival_months.loc[4,202308]=df_festival_month.query("region=='강원도'")['08'].sum()
df_festival_months.loc[0,202309]=df_festival_month.query("region=='인천'")['09'].sum()
df_festival_months.loc[1,202309]=df_festival_month.query("region=='부산'")['09'].sum()
df_festival_months.loc[2,202309]=df_festival_month.query("region=='서울'")['09'].sum()
df_festival_months.loc[3,202309]=df_festival_month.query("region=='제주'")['09'].sum()
df_festival_months.loc[4,202309]=df_festival_month.query("region=='강원도'")['09'].sum()
df_festival_months.loc[0,202310]=df_festival_month.query("region=='인천'")['10'].sum()
df_festival_months.loc[1,202310]=df_festival_month.query("region=='부산'")['10'].sum()
df_festival_months.loc[2,202310]=df_festival_month.query("region=='서울'")['10'].sum()
df_festival_months.loc[3,202310]=df_festival_month.query("region=='제주'")['10'].sum()
df_festival_months.loc[4,202310]=df_festival_month.query("region=='강원도'")['10'].sum()
df_festival_months.loc[0,202311]=df_festival_month.query("region=='인천'")['11'].sum()
df_festival_months.loc[1,202311]=df_festival_month.query("region=='부산'")['11'].sum()
df_festival_months.loc[2,202311]=df_festival_month.query("region=='서울'")['11'].sum()
df_festival_months.loc[3,202311]=df_festival_month.query("region=='제주'")['11'].sum()
df_festival_months.loc[4,202311]=df_festival_month.query("region=='강원도'")['11'].sum()
df_festival_months.loc[0,202312]=df_festival_month.query("region=='인천'")['12'].sum()
df_festival_months.loc[1,202312]=df_festival_month.query("region=='부산'")['12'].sum()
df_festival_months.loc[2,202312]=df_festival_month.query("region=='서울'")['12'].sum()
df_festival_months.loc[3,202312]=df_festival_month.query("region=='제주'")['12'].sum()
df_festival_months.loc[4,202312]=df_festival_month.query("region=='강원도'")['12'].sum()
df_festival_months.set_index('지역구분', inplace=True)
df_festival_months.loc['인천']

202301    0.0
202302    0.0
202303    0.0
202304    2.0
202305    2.0
202306    1.0
202307    0.0
202308    2.0
202309    5.0
202310    6.0
202311    1.0
202312    2.0
Name: 인천, dtype: float64

In [199]:
df_festival_months.index

Index(['인천', '부산', '서울', '제주', '강원도'], dtype='object', name='지역구분')

In [125]:
pivot_festival_month

,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312
region,,,,,,,,,,,,
강원,4.0,2.0,1.0,2.0,5.0,2.0,2.0,1.0,5.0,5.0,1.0,0.0
부산,2.0,3.0,1.0,2.0,4.0,1.0,2.0,4.0,2.0,18.0,5.0,6.0
서울,0.0,0.0,1.0,8.0,10.0,5.0,2.0,3.0,11.0,23.0,4.0,2.0
인천,0.0,0.0,0.0,2.0,2.0,1.0,0.0,2.0,5.0,6.0,1.0,2.0
제주,6.0,8.0,17.0,22.0,23.0,29.0,25.0,28.0,32.0,34.0,15.0,4.0


In [196]:
pivot_visitor.index

Index(['강원', '부산', '서울', '인천', '제주'], dtype='object', name='지역구분')

In [78]:
pivot_festival_month = pivot_festival_month.rename_axis('지역구분')

In [79]:
merged_df = pd.merge(pivot_festival_month, pivot_visitor, on='지역구분') # 두 피벗 테이블을 합쳤는데 쓸모가 있을까?

### 상관분석

#### 축제의 지역별 정규성 파악

In [195]:
pivot_visitor

기준년월,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312
지역구분,,,,,,,,,,,,
강원,12613168,11183755,12150911,11164195,12499622,12291416,14682347,17103945,13520176,13928972,9878631,11320438
부산,13180063,11238505,13159913,11489345,12455221,11702524,12825366,13322368,13666837,12641711,11626722,12946350
서울,47255715,47090565,54980858,45025765,49506461,47768488,50292484,49277340,51928425,51118153,49451765,52547645
인천,14321833,13187354,15620369,13830096,15906026,14338822,15492589,15449316,16422889,16131144,14341875,15642889
제주,5377591,4935915,5274889,5123197,5208883,5059521,5336175,5982112,4788081,5614039,4672499,4638646


In [80]:
pivot_visitor.loc["강원"]

기준년월
202301    12613168.0
202302    11183755.0
202303    12150911.0
202304    11164195.0
202305    12499622.0
202306    12291416.0
202307    14682347.0
202308    17103945.0
202309    13520176.0
202310    13928972.0
202311     9878631.0
202312    11320438.0
Name: 강원, dtype: float64

In [81]:
stats.shapiro(pivot_visitor.loc["강원"]) # pvalue > 0.05이므로 정규분포를 따른다.

ShapiroResult(statistic=0.9382747644577106, pvalue=0.4760507773414123)

In [82]:
round(9.314088625374506e-07,5)

0.0

In [83]:
stats.shapiro(pivot_visitor.loc["제주"]) # pvalue > 0.05이므로 정규분포를 따른다.

ShapiroResult(statistic=0.9618033569729677, pvalue=0.8092344673462573)

In [84]:
stats.shapiro(pivot_visitor.loc["서울"]) # pvalue > 0.05이므로 정규분포를 따른다.

ShapiroResult(statistic=0.9856587827031315, pvalue=0.9973136453905183)

In [87]:
stats.shapiro(pivot_visitor.loc["부산"])# pvalue > 0.05이므로 정규분포를 따른다.

ShapiroResult(statistic=0.9198427916245954, pvalue=0.28461342141298596)

In [86]:
stats.shapiro(pivot_visitor.loc["인천"])# pvalue > 0.05이므로 정규분포를 따른다.

ShapiroResult(statistic=0.9248841521619882, pvalue=0.32901922814749984)

### 관광객의 지역별 정규분포 확인

In [205]:
stats.shapiro(df_festival_months.loc["강원도"]) # pvalue < 0.05이므로 비정규분포를 따른다.

ShapiroResult(statistic=0.8587074061052631, pvalue=0.04709942700585504)

In [206]:
stats.shapiro(df_festival_months.loc["제주"]) # pvalue > 0.05이므로 정규분포를 따른다.

ShapiroResult(statistic=0.9354095989679595, pvalue=0.4409841177509154)

In [207]:
stats.shapiro(df_festival_months.loc["서울"]) # pvalue < 0.05이므로 비정규분포를 따른다.

ShapiroResult(statistic=0.808361839081397, pvalue=0.01168008702619159)

In [208]:
stats.shapiro(df_festival_months.loc["부산"]) # pvalue < 0.05이므로 비정규분포를 따른다.

ShapiroResult(statistic=0.6321704300464863, pvalue=0.00020166224234890252)

In [209]:
stats.shapiro(df_festival_months.loc["인천"]) # pvalue < 0.05이므로 비정규분포를 따른다.

ShapiroResult(statistic=0.8119591133313422, pvalue=0.012854813577675782)

### 관광객과 축제 수의 상관관계 확인

In [211]:
# pvalue < 0.05 : 상관관계가 있음
stats.spearmanr(pivot_visitor.loc["인천"],df_festival_months.loc["인천"])

SignificanceResult(statistic=0.6066113300237268, pvalue=0.036500298354033275)

In [212]:
# pvalue < 0.05 : 상관관계가 있음
stats.spearmanr(pivot_visitor.loc["부산"],df_festival_months.loc["부산"])

SignificanceResult(statistic=-0.10716746474310215, pvalue=0.7402646261347783)

In [211]:
# pvalue < 0.05 : 상관관계가 있음
stats.spearmanr(pivot_visitor.loc["서울"],df_festival_months.loc["서울
"])

SignificanceResult(statistic=0.6066113300237268, pvalue=0.036500298354033275)

In [214]:
# pvalue < 0.05 : 상관관계가 있음
stats.spearmanr(pivot_visitor.loc["강원"],df_festival_months.loc["강원도"])

SignificanceResult(statistic=0.37564959566670514, pvalue=0.22883887632115527)

In [215]:
# pvalue < 0.05 : 상관관계가 있음
stats.pearsonr(pivot_visitor.loc["제주"],df_festival_months.loc["제주"])

PearsonRResult(statistic=0.4215735822815469, pvalue=0.17226743750900186)